In [7]:
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pyvista as pv
import trimesh
from copy import deepcopy
%matplotlib widget

### Define a function to fix the normals of both the sphere and lobes (courtesy of Joshua Soderholm)

In [8]:
def recalculate_normals_from_centroid(mesh):
    """
    Recalculate all normals to point outward from the mesh centroid.
    Handles triangular, quad, and mixed polygon meshes.
    
    Parameters
    ----------
    mesh : pv.PolyData
        The mesh to process
        
    Returns
    -------
    mesh : pv.PolyData
        The mesh with corrected normals
    """
    # Get the centroid of the entire mesh
    centroid = mesh.center
    
    # Get cell centers
    cell_centers = mesh.cell_centers().points
    
    # Calculate vectors from centroid to each cell center
    outward_vectors = cell_centers - centroid
    
    # Normalize these vectors to get desired normal directions
    desired_normals = outward_vectors / np.linalg.norm(outward_vectors, axis=1, keepdims=True)
    
    # Compute the current normals
    mesh.compute_normals(cell_normals=True, point_normals=False, inplace=True)
    current_normals = mesh.cell_data['Normals']
    
    # Check which normals need flipping
    dot_products = np.sum(current_normals * desired_normals, axis=1)
    needs_flipping = dot_products < 0
    
    # Process faces - need to handle variable polygon sizes
    faces = mesh.faces.copy()
    n_cells = mesh.n_cells
    
    # Parse faces array and flip as needed
    idx = 0
    for cell_id in range(n_cells):
        # First value is number of points in this face
        n_points = faces[idx]
        
        # Get the vertex indices for this face
        vertex_indices = faces[idx+1:idx+1+n_points]
        
        # If this face needs flipping, reverse the vertex order
        if needs_flipping[cell_id]:
            faces[idx+1:idx+1+n_points] = vertex_indices[::-1]
        
        # Move to next face
        idx += n_points + 1
    
    # Update the mesh faces
    mesh.faces = faces
    
    # Recompute normals after flipping
    mesh.compute_normals(cell_normals=True, point_normals=True, consistent_normals=True)
    
    print(f"Flipped {needs_flipping.sum()} out of {len(needs_flipping)} faces")
    
    return mesh

### Create a set of equidistant points for a sphere

In [9]:
# Create a range of equidistant points on a sphere using Deserno 2004,
# but with correction that points are first determined on unit sphere, 
# then expanded by conversion to cartesian coordinates

r = 1 #radius in mm
N=5000 #goal of number points
sphere_points=[]
n_count = 0 # actual number of points


#a = 4*np.pi*r**2./ N
a = 4*np.pi / N
d = np.sqrt(a)
M_theta = np.round(np.pi/d)
d_theta = np.pi/M_theta
d_phi = a/d_theta
for m in np.arange(0,M_theta):
    theta = np.pi*(m+0.5) / M_theta
    M_phi = np.round(2*np.pi*np.sin(theta)/d_phi)
    for n in np.arange(0,M_phi):
        phi = 2*np.pi*n/M_phi
        x = r * np.sin(theta)*np.cos(phi)
        y = r * np.sin(theta)*np.sin(phi)
        z = r * np.cos(theta)
        sphere_points.append((x,y,z))
        n_count += 1

print (n_count)
sphere_points = np.array(sphere_points)

4999


### Add equally spaced lobes to the sphere surface

In [10]:
n_lobes = 50 #goal number of lobes
lobe_centers = [] #just the center points
lobes = [] #successive sets of points, including circles around center points, one for each lobe
lobe_count = 0 #actual number of lobes
lobe_thetas = []
lobe_phis = []

#get the spherical coordinates (on a unit sphere) of all lobe centers
#basically, repeat the stuff in previous cell but for fewer points
a = 4*np.pi / n_lobes
d = np.sqrt(a)
M_theta = np.round(np.pi/d)
#d_theta = np.pi/M_theta
d_theta = 0.95*np.pi/M_theta #seemed to crowd at the poles, so this pulls it away a bit more
d_phi = a/d_theta
for m in np.arange(0,M_theta):
    #theta = np.pi*(m+0.5) / M_theta
    theta = 0.95*np.pi*(m+0.55) / M_theta
    #theta = np.pi*(m+0.55) / M_theta
    M_phi = np.round(2*np.pi*np.sin(theta)/d_phi)
    for n in np.arange(0,M_phi):
        phi = 2*np.pi*n/M_phi
        lobe_thetas.append(theta)
        lobe_phis.append(phi)
        x = r * np.sin(theta)*np.cos(phi)
        y = r * np.sin(theta)*np.sin(phi)
        z = r * np.cos(theta)
        lobe_centers.append((x,y,z))
        lobe_count += 1

#now calculate circles around each lobe point, at a central angle of alpha
# following https://math.stackexchange.com/questions/643130/circle-on-sphere
# Ah ha! But note their phi and theta angles are switched from the Deserno 2004 paper
alpha_start = 0.45*np.minimum(d_theta, d_phi)
radius = 1. #set to sphere radius to start
num_t_at_start = 200
circ_at_start = 2*np.pi*alpha_start
dt = circ_at_start / num_t_at_start

for phi, theta, lobe_center in zip(lobe_phis, lobe_thetas, lobe_centers):
    beta = theta
    gamma = np.pi - phi
    lobe_points = []
    
    #cycle through successively smaller alphas, setting the radius/height increasingly higher
    # each time
    alphas = np.linspace(alpha_start, 0, 200)
    #lobe_heights = radius + np.sin(np.pi/2 * np.linspace(0,0.5,10)) #equation too sharp
    lobe_heights = radius + -(np.cos(np.pi*np.linspace(0,1.0,200))-1)/4  #followed https://easings.net/#easeInOutSine
    first_time_through = True
    for alpha, lobe_height in zip(alphas[:-1], lobe_heights[:-1]):
    
        #calculate number of t needed for this alpha
        circ_here = 2*np.pi*alpha
        num_t = int(np.round(circ_here/dt))
        t = np.linspace(0,2*np.pi,num_t)
        #print (alpha, circ_here, num_t)

        #cartesian coordinates for each point in the circle 
        xs = lobe_height*(np.sin(alpha)*np.cos(beta)*np.cos(gamma)*np.cos(t) + 
                            np.sin(alpha)*np.sin(gamma)*np.sin(t)-
                            np.cos(alpha)*np.sin(beta)*np.cos(gamma))
        ys = lobe_height*(-np.sin(alpha)*np.cos(beta)*np.sin(gamma)*np.cos(t) + 
                            np.sin(alpha)*np.cos(gamma)*np.sin(t)+
                            np.cos(alpha)*np.sin(beta)*np.sin(gamma))
        zs = lobe_height*(np.sin(alpha)*np.sin(beta)*np.cos(t)+np.cos(alpha)*np.cos(beta))
        
        # if first time through, calculate great circle distance from center point to all points in circle
        # (should all be the same)
        if first_time_through:
            #gcd between circle and center point on unit sphere is just the angle between them - alpha!
            #cartesian coordinates calculation if you want to prove it, below
            # gcd = []
            # for thisx,thisy,thisz in zip(xs,ys,zs):
            #     a = np.array((thisx,thisy,thisz))
            #     b = np.array(lobe_centers[0])
            #     a_dot_b = np.dot(a,b)
            #     angle = np.arccos(a_dot_b / ( (a[0]**2+a[1]**2+a[2]**2)**0.5 * (b[0]**2+b[1]**2+b[2]**2)**0.5 ) )
            #     gcd.append(r*angle)
            gcd = alpha
            
            #now, determine which points in sphere points array are closer to center point than that.
            #remove them, as they will become "inside" points underneath the lobe once surfaces are merged.
            a = sphere_points
            b = np.array(lobe_center)
            a_dot_b = np.dot(a,b)
            angle = np.arccos(a_dot_b / ( (a[:,0]**2+a[:,1]**2+a[:,2]**2)**0.5 * (b[0]**2+b[1]**2+b[2]**2)**0.5 ) )
            points_gcds = r*angle
            inside_circle = points_gcds < gcd
            sphere_points = np.delete(sphere_points,inside_circle,axis=0)

        #last point is right at alpha = 0
        alpha = alphas[-1]
        lobe_height = lobe_heights[-1]
        xs = np.append(xs, lobe_height*(np.sin(theta)*np.cos(phi))) #use the x,y,z location of lobe center but add lobe_height
        ys = np.append(ys, lobe_height*(np.sin(theta)*np.sin(phi)))
        zs = np.append(zs, lobe_height*(np.cos(theta)))
        
        dum = [lobe_points.append((i,j,k)) for i,j,k in zip(xs,ys,zs)]
        first_time_through = False
    
    lobes.append(lobe_points)

print (len(lobes))
print (len(lobe_centers))
print (sphere_points.shape)



51
51
(1785, 3)


### Reconstruct a surface from points in the point cloud

In [22]:
lobe_centers = np.array(lobe_centers)
lobe_cloud = pv.PolyData(lobe_centers)
sphere_cloud = pv.PolyData(sphere_points)


#calculate a separate surface for each lobe
lobe_surfs = []
#all_lobe_points = []
for lobe in lobes:
    #all_lobe_points = all_lobe_points+lobe
    lobe_points = np.array(lobe)
    lobe_point_cloud = pv.PolyData(lobe_points)
    lobe_surfs.append(lobe_point_cloud.reconstruct_surface(sample_spacing=0.05))

#make a point cloub of all the lobe points
#all_lobe_points = np.array(all_lobe_points)
#lobe_point_cloud = pv.PolyData(all_lobe_points)

#add our surfaces together
sphere_surf = sphere_cloud.reconstruct_surface()
total_surf = deepcopy(sphere_surf)
for lobe_surf in lobe_surfs:
    total_surf = total_surf + lobe_surf 

# Fix the surface normals to make them all point outward
cell_normals_recal = recalculate_normals_from_centroid(total_surf)

#and plot, by individual lobe and then combined surface    
pv.global_theme.color_cycler = 'default'
pl = pv.Plotter(shape=(1, 2))
pl.subplot(0, 0)
_ = pl.add_mesh(sphere_surf)
#_ = pl.add_mesh(lobe_point_cloud,color='k')
for lobe_surf in lobe_surfs:
    _ = pl.add_mesh(lobe_surf)
    
pl.subplot(0, 1)
_ = pl.add_mesh(total_surf,color='blue')
pl.show()



Flipped 27268 out of 40213 faces


Widget(value='<iframe src="http://localhost:55363/index.html?ui=P_0x30ce224e0_5&reconnect=auto" class="pyvista…

### Write mesh out to STL file

In [23]:
total_surf.save('equimesh_50lobe.stl')


#now plot again
mesh = trimesh.load_mesh("equimesh_50lobe.stl")
# Show the mesh (opens in a window)
mesh.show()


In [24]:
#mesh quality
print(f"- Is watertight: {mesh.is_watertight}")
print(f"- Volume: {mesh.volume:.4f}")
print(f"- Surface area: {mesh.area:.4f}")




- Is watertight: False
- Volume: 11.1792
- Surface area: 45.1668


### Try to fix the small holes that make the shape not watertight

In [63]:
def fill_all_hole_loops(mesh, holes):
    """
    Fill all hole loops on a triangulated PyVista mesh by creating fan triangles
    from each loop's centroid to its boundary vertices.
    
    Parameters
    ----------
    mesh : pyvista.PolyData
        Triangulated input mesh (will not be modified in-place).
    holes : pyvista.PolyData
        Lines polydata returned by MeshFix.extract_holes() (edge format).
    
    Returns
    -------
    pyvista.PolyData
        New mesh with all holes filled (triangles appended).
    """
    import numpy as np
    import pyvista as pv
    
    if holes is None or holes.n_points == 0 or holes.n_cells == 0:
        return mesh.copy()
    
    # Parse holes.lines to extract edges (2-point line segments)
    lines = holes.lines.astype(int)
    edges = []
    idx = 0
    while idx < len(lines):
        n = int(lines[idx])
        if n == 2:  # edge
            p0, p1 = lines[idx+1], lines[idx+2]
            edges.append((p0, p1))
        idx += n + 1
    
    if not edges:
        return mesh.copy()
    
    # Reconstruct closed loops from edges
    # Build adjacency: for each point, what points does it connect to?
    from collections import defaultdict
    adj = defaultdict(list)
    for p0, p1 in edges:
        adj[p0].append(p1)
        adj[p1].append(p0)
    
    # Extract loops by following edges
    loops = []
    visited_edges = set()
    
    for start_p, neighbors in adj.items():
        for next_p in neighbors:
            edge_key = (min(start_p, next_p), max(start_p, next_p))
            if edge_key in visited_edges:
                continue
            
            # Trace loop starting from this edge
            loop = [start_p, next_p]
            current = next_p
            prev = start_p
            visited_edges.add(edge_key)
            
            while current != start_p:
                # Find next neighbor (not the one we came from)
                next_neighbors = [p for p in adj[current] if p != prev]
                if not next_neighbors:
                    break  # Dead end, not a closed loop
                
                next_p = next_neighbors[0]
                edge_key = (min(current, next_p), max(current, next_p))
                visited_edges.add(edge_key)
                
                if next_p == start_p:
                    # Loop closed
                    if len(loop) >= 3:
                        loops.append(np.array(loop))
                    break
                else:
                    loop.append(next_p)
                    prev = current
                    current = next_p
    
    if not loops:
        print("No closed loops found in hole edges")
        return mesh.copy()
    
    print(f"Found {len(loops)} hole loops")
    
    # Copy original mesh data
    orig_pts = mesh.points.copy()
    orig_faces = mesh.faces.astype(int)
    
    # Convert existing faces to explicit triangle list. Not necessary if already triangulated.
    tris = []
    j = 0
    while j < len(orig_faces):
        nv = int(orig_faces[j])
        verts = orig_faces[j + 1: j + 1 + nv].tolist()
        if nv == 3:
            tris.append(verts)
        else:
            # fan-triangulate polygon
            for k in range(1, nv - 1):
                tris.append([verts[0], verts[k], verts[k + 1]])
        j += nv + 1
    
    tris = np.array(tris, dtype=int)
    
    # Start with original mesh points, then append hole boundary points and centroids
    new_points = orig_pts.tolist()
    new_tris = tris.tolist()
    
    # Map from holes.points indices to new_points indices
    hole_pts_offset = len(new_points)
    for pt in holes.points:
        new_points.append(pt.tolist())
    
    # Fill each hole loop with fan triangles to its centroid
    for loop_idx, loop in enumerate(loops):
        # loop contains indices into holes.points
        # Map them to indices in new_points
        loop_indices_in_new = [hole_pts_offset + idx for idx in loop]
        
        # Compute centroid from actual hole boundary points
        coords = holes.points[loop]
        centroid = coords.mean(axis=0)
        centroid_idx = len(new_points)
        new_points.append(centroid.tolist())
        
        # Create fan triangles: (loop[i], loop[i+1], centroid) for each edge
        L = len(loop_indices_in_new)
        
        for k in range(L):
            i0 = loop_indices_in_new[k]
            i1 = loop_indices_in_new[(k + 1) % L]
            new_tris.append([i0, i1, centroid_idx])
    
    # Build VTK faces array from triangles: [3, i0, i1, i2, 3, j0, j1, j2, ...]
    new_tris = np.array(new_tris, dtype=np.int64)
    
    # Create faces array: prepend count (3) to each triangle
    faces_list = []
    for tri in new_tris:
        faces_list.append(3)
        faces_list.extend(tri)
    
    faces_vtk = np.array(faces_list, dtype=np.int64)
    
    # Create and return new mesh
    new_pts_arr = np.array(new_points, dtype=float)
    print(f"Created mesh with {len(new_tris)} triangles, {len(new_pts_arr)} points")
    new_mesh = pv.PolyData(new_pts_arr, faces_vtk)
    
    # Recompute normals 
    new_mesh.compute_normals(cell_normals=True, point_normals=True, consistent_normals=True)
    
    return new_mesh


In [68]:
import pymeshfix as mf

tri_surf = total_surf.triangulate()
mesh_to_fix = mf.MeshFix(tri_surf)
holes = mesh_to_fix.extract_holes()

# Show holes before filling
p = pv.Plotter()
p.add_mesh(tri_surf, color='blue')
p.add_mesh(holes, color='red', line_width=8)
p.show()

# Fill all holes
filled = fill_all_hole_loops(tri_surf, holes)
print(f"Original mesh: {tri_surf.n_points} points, {tri_surf.n_cells} cells")
print(f"Filled mesh: {filled.n_points} points, {filled.n_cells} cells")

# Check if watertight (convert to trimesh to access is_watertight)
filled_trimesh = trimesh.Trimesh(vertices=filled.points, faces=filled.faces.reshape(-1, 4)[:, 1:])
print(f"Is filled mesh watertight? {filled_trimesh.is_watertight}")

# Visualize result
#filled.plot()


Widget(value='<iframe src="http://localhost:55363/index.html?ui=P_0x3a19d8d40_51&reconnect=auto" class="pyvist…

Found 51 hole loops
Created mesh with 43054 triangles, 24472 points
Original mesh: 21580 points, 40213 cells
Filled mesh: 24472 points, 43054 cells
Is filled mesh watertight? True


In [71]:

# Diagnostic: analyze what holes remain
print("\n=== DIAGNOSTIC ===")
print(f"Original mesh is_watertight: {mesh.is_watertight}")
print(f"Filled mesh (trimesh) is_watertight: {filled_trimesh.is_watertight}")
print(f"Filled mesh volume: {filled_trimesh.volume:.6f}")

# Check if there are still holes in the filled mesh
mesh_to_fix_after = mf.MeshFix(filled)
holes_after = mesh_to_fix_after.extract_holes()

print(f"\nHoles before filling: {holes.n_cells} hole segments")
print(f"Holes after filling: {holes_after.n_cells} hole segments")

# The key question: is it actually watertight for practical purposes?
if filled_trimesh.is_watertight:
    print("\n✓ SUCCESS: Mesh is watertight according to trimesh!")
    print("  (The remaining holes detected by MeshFix are likely topological artifacts)")
    print("  Your mesh is ready for use.")
    
    # Save the watertight mesh
    filled_trimesh.export('equimesh_50lobe_watertight.stl')
    print("  Saved to: equimesh_50lobe_watertight.stl")
else:
    print("\n✗ Mesh is still not watertight")
    # Visualize remaining holes if any
    if holes_after.n_cells > 0:
        print("Remaining holes detected! Visualizing...")
        p2 = pv.Plotter()
        p2.add_mesh(filled, color='blue', opacity=0.7)
        p2.add_mesh(holes_after, color='red', line_width=10)
        p2.show()



=== DIAGNOSTIC ===
Original mesh is_watertight: False
Filled mesh (trimesh) is_watertight: True
Filled mesh volume: 16.076165

Holes before filling: 2841 hole segments
Holes after filling: 5682 hole segments

✓ SUCCESS: Mesh is watertight according to trimesh!
  (The remaining holes detected by MeshFix are likely topological artifacts)
  Your mesh is ready for use.
  Saved to: equimesh_50lobe_watertight.stl
